In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
device_gpu = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import torch.nn.functional as F
from sklearn.model_selection import train_test_split 
import copy

In [ ]:
import nltk 
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from nltk.tokenize import punkt
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_en=set(stopwords.words('english'))
stop_es=set(stopwords.words('spanish'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
print(stop_es)

{'habiendo', 'tendrían', 'tengan', 'fuera', 'otros', 'mío', 'tienes', 'estás', 'sentida', 'esos', 'hubiste', 'sobre', 'esto', 'contra', 'hasta', 'habrán', 'vuestra', 'cual', 'serían', 'ella', 'todo', 'nada', 'estés', 'están', 'estuviesen', 'hayáis', 'hubo', 'hubimos', 'tuvimos', 'no', 'ni', 'mis', 'su', 'estuve', 'tenga', 'ya', 'fueron', 'muy', 'suyos', 'esas', 'tenida', 'los', 'vuestro', 'sintiendo', 'otro', 'hayas', 'el', 'era', 'estará', 'a', 'habría', 'suya', 'fui', 'estado', 'sea', 'estuvo', 'tendrá', 'e', 'mucho', 'seré', 'nuestras', 'estuviese', 'serías', 'estuvisteis', 'como', 'estuvieses', 'habríamos', 'pero', 'tendrías', 'estéis', 'somos', 'tuyo', 'estuvieron', 'estamos', 'tendré', 'estemos', 'es', 'fuiste', 'habríais', 'sentido', 'lo', 'más', 'te', 'esta', 'algunos', 'eran', 'y', 'tenían', 'o', 'tendrás', 'tendrán', 'otra', 'qué', 'hubieseis', 'tuvieseis', 'vuestros', 'tuviéramos', 'estando', 'vosotras', 'tuvo', 'estas', 'estuvieran', 'eras', 'tus', 'él', 'éramos', 'por', 'n

In [ ]:
europarl_en = open('/content/drive/My Drive/Projects/es-en/es-en/europarl-v7.es-en.en', encoding='utf-8').read().split('\n')
europarl_es = open('/content/drive/My Drive/Projects/es-en/es-en 2/europarl-v7.es-en.es', encoding='utf-8').read().split('\n')

In [ ]:
print(len(europarl_en))

1965735


In [ ]:
print(europarl_es[:5])

['Reanudación del período de sesiones', 'Declaro reanudado el período de sesiones del Parlamento Europeo, interrumpido el viernes 17 de diciembre pasado, y reitero a Sus Señorías mi deseo de que hayan tenido unas buenas vacaciones.', 'Como todos han podido comprobar, el gran "efecto del año 2000" no se ha producido. En cambio, los ciudadanos de varios de nuestros países han sido víctimas de catástrofes naturales verdaderamente terribles.', 'Sus Señorías han solicitado un debate sobre el tema para los próximos días, en el curso de este período de sesiones.', 'A la espera de que se produzca, de acuerdo con muchos colegas que me lo han pedido, pido que hagamos un minuto de silencio en memoria de todas las víctimas de las tormentas, en los distintos países de la Unión Europea afectados.']


In [ ]:
!python -m spacy download en
!python -m spacy download es

In [ ]:
import spacy
import torchtext
from torchtext.data import Field, BucketIterator, TabularDataset
en=spacy.load('en')
es=spacy.load('es')


In [ ]:
def tokenize_en(sentence):
    return [tok.text for tok in en.tokenizer(sentence)]
def tokenize_es(sentence):
    return [tok.text for tok in es.tokenizer(sentence)]
EN_TEXT = Field(tokenize=tokenize_en)
ES_TEXT = Field(tokenize=tokenize_es, init_token = "<sos>", eos_token = "<eos>")

In [ ]:
raw_data={"English":[line for line in europarl_en],"Spanish":[line for line in europarl_es]}
df=pd.DataFrame(raw_data,columns=["English","Spanish"])
df.head()

,English,Spanish
0,Resumption of the session,Reanudación del período de sesiones
1,I declare resumed the session of the European ...,Declaro reanudado el período de sesiones del P...
2,"Although, as you will have seen, the dreaded '...","Como todos han podido comprobar, el gran ""efec..."
3,You have requested a debate on this subject in...,Sus Señorías han solicitado un debate sobre el...
4,"In the meantime, I should like to observe a mi...","A la espera de que se produzca, de acuerdo con..."


In [ ]:
df.shape

(1965735, 2)

In [ ]:
df['eng_len'] = df['English'].str.count(' ')
df['es_len'] = df['Spanish'].str.count(' ')
df=df.query('es_len < 80 & eng_len < 80')


In [ ]:
train,val=train_test_split(df,test_size=0.1)
#df.to_csv('/drive/My Drive/folder_name/name_csv_file.csv')
train.to_csv("/content/drive/My Drive/Projects/es-en/train.csv",index=False)
val.to_csv("/content/drive/My Drive/Projects/es-en/val.csv",index=False)

In [ ]:
data_fields=[('English',EN_TEXT),("Spanish",ES_TEXT)]
data_fields

[('English', <torchtext.data.field.Field at 0x7f5c237e66d8>),
 ('Spanish', <torchtext.data.field.Field at 0x7f5c237e6588>)]

In [ ]:
train=TabularDataset(path='/content/drive/My Drive/Projects/es-en/train.csv',format='csv',fields=data_fields)
val=TabularDataset(path='/content/drive/My Drive/Projects/es-en/val.csv',format='csv',fields=data_fields)

In [ ]:
type(train)

torchtext.data.dataset.TabularDataset

In [ ]:
pre_trained_vector_type = 'glove.6B.200d'
EN_TEXT.build_vocab(train,val,pre_trained_vector_type)
ES_TEXT.build_vocab(train,val,pre_trained_vector_type)

In [ ]:
from torchtext import data
global max_src_in_batch, max_tgt_in_batch
def batch_size_fn(new, count, sofar):
    "Keep augmenting batch and calculate total number of tokens + padding."
    global max_src_in_batch, max_tgt_in_batch
    if count == 1:
        max_src_in_batch = 0
        max_tgt_in_batch = 0
    max_src_in_batch = max(max_src_in_batch,  len(new.English))
    max_tgt_in_batch = max(max_tgt_in_batch,  len(new.Spanish) + 2)
    src_elements = count * max_src_in_batch
    tgt_elements = count * max_tgt_in_batch
    return max(src_elements, tgt_elements)
class MyIterator(data.Iterator):
    def create_batches(self):
        if self.train:
            def pool(d, random_shuffler):
                for p in data.batch(d, self.batch_size * 100):
                    p_batch = data.batch(
                        sorted(p, key=self.sort_key),
                        self.batch_size, self.batch_size_fn)
                    for b in random_shuffler(list(p_batch)):
                        yield b
            self.batches = pool(self.data(), self.random_shuffler)
            
        else:
            self.batches = []
            for b in data.batch(self.data(), self.batch_size,
                                          self.batch_size_fn):
                self.batches.append(sorted(b, key=self.sort_key))

In [ ]:
train_iter = MyIterator(train, batch_size=1300, device=device_gpu,
                        repeat=False, sort_key= lambda x:
                        (len(x.English), len(x.Spanish)),
                        batch_size_fn=batch_size_fn, train=True,
                        shuffle=True)
batch=next(iter(train_iter))
print(batch.English)
type(batch.English)

In [ ]:
batch.Spanish.shape

torch.Size([28, 46])

In [ ]:
a=[]
b=[]
for i in range(len(EN_TEXT.vocab)):
    a.append(EN_TEXT.vocab.itos[i])
for i in range(len(ES_TEXT.vocab)):
    b.append(ES_TEXT.vocab.itos[i])
type(a)

list

In [ ]:
EN_TEXT.vocab.itos[3]

','

In [ ]:
mix=pd.DataFrame()
mix1=pd.DataFrame()
mix["EN_TEXT"]=a
mix1["ES_TEXT"]=b

In [ ]:
class Embedder(nn.Module):
    def __init__(self, vocab_size, d_model):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, d_model)
    def forward(self, x):
        return self.embed(x)

In [ ]:
import math
class PositionalEncoder(nn.Module):
    def __init__(self, d_model, max_seq_len = 80):
        super().__init__()
        self.d_model = d_model
        # create constant 'pe' matrix with values dependant on 
        # pos and i
        pe = torch.zeros(max_seq_len, d_model)
        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe[pos, i] =math.sin(pos / (10000 ** ((2 * i)/d_model)))
                pe[pos, i + 1] =math.cos(pos / (10000 ** ((2 * (i + 1))/d_model)))
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
    def forward(self, x):
        # make embeddings relatively larger
        x = x * math.sqrt(self.d_model)
        #add constant to embedding
        seq_len = x.size(1)
        x = x + Variable(self.pe[:,:seq_len], requires_grad=False).cuda()
        return x

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, heads, d_model, dropout = 0.1):
        super().__init__()
        self.d_model = d_model
        self.d_k = d_model // heads
        self.h = heads
        self.q_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
        self.out = nn.Linear(d_model, d_model)
    def forward(self, q, k, v, mask=None):
        bs = q.size(0)
        # perform linear operation and split into h heads
        k = self.k_linear(k).view(bs, -1, self.h, self.d_k)
        q = self.q_linear(q).view(bs, -1, self.h, self.d_k)
        v = self.v_linear(v).view(bs, -1, self.h, self.d_k)
        # transpose to get dimensions bs * h * sl * d_model
        k = k.transpose(1,2)
        q = q.transpose(1,2)
        v = v.transpose(1,2)
        # calculate attention using function we will define next
        scores = attention(q, k, v, self.d_k, mask, self.dropout)
        # concatenate heads and put through final linear layer
        concat = scores.transpose(1,2).contiguous().view(bs, -1, self.d_model)
        output = self.out(concat)
        return output

In [ ]:
def attention(q, k, v, d_k, mask=None, dropout=None):
    scores = torch.matmul(q, k.transpose(-2, -1)) /  math.sqrt(d_k)
    if mask is not None:
        mask = mask.unsqueeze(1)
        scores = scores.masked_fill(mask == 0, -1e9)
    scores = F.softmax(scores, dim=-1)
    if dropout is not None:
        scores = dropout(scores)
    output = torch.matmul(scores, v)
    return output

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff=2048, dropout = 0.1):
        super().__init__() 
        # We set d_ff as a default to 2048
        self.linear_1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model)
    def forward(self, x):
        x = self.dropout(F.relu(self.linear_1(x)))
        x = self.linear_2(x)
        return x

In [ ]:
class Norm(nn.Module):
    def __init__(self, d_model, eps = 1e-6):
        super().__init__()
        self.size = d_model
        # create two learnable parameters to calibrate normalisation
        self.alpha = nn.Parameter(torch.ones(self.size))
        self.bias = nn.Parameter(torch.zeros(self.size))
        self.eps = eps
    def forward(self, x):
        norm = self.alpha * (x - x.mean(dim=-1, keepdim=True))/(x.std(dim=-1, keepdim=True) + self.eps) + self.bias
        return norm

In [ ]:
# build an encoder layer with one multi-head attention layer and one # feed-forward layer
class EncoderLayer(nn.Module):
    def __init__(self, d_model, heads, dropout = 0.1):
        super().__init__()
        self.norm_1 = Norm(d_model)
        self.norm_2 = Norm(d_model)
        self.attn = MultiHeadAttention(heads, d_model)
        self.ff = FeedForward(d_model)
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)
    def forward(self, x, mask):
        x2 = self.norm_1(x)
        x = x + self.dropout_1(self.attn(x2,x2,x2,mask))
        x2 = self.norm_2(x)
        x = x + self.dropout_2(self.ff(x2))
        return x
# build a decoder layer with two multi-head attention layers and
# one feed-forward layer
class DecoderLayer(nn.Module):
    def __init__(self, d_model, heads, dropout=0.1):
        super().__init__()
        self.norm_1 = Norm(d_model)
        self.norm_2 = Norm(d_model)
        self.norm_3 = Norm(d_model)
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)
        self.dropout_3 = nn.Dropout(dropout)
        self.attn_1 = MultiHeadAttention(heads, d_model)
        self.attn_2 = MultiHeadAttention(heads, d_model)
        self.ff = FeedForward(d_model).to(device_gpu)
    def forward(self, x, e_outputs, src_mask, trg_mask):
        x2 = self.norm_1(x)
        x = x + self.dropout_1(self.attn_1(x2, x2, x2, trg_mask))
        x2 = self.norm_2(x)
        x = x + self.dropout_2(self.attn_2(x2, e_outputs, e_outputs,src_mask))
        x2 = self.norm_3(x)
        x = x + self.dropout_3(self.ff(x2))
        return x
# We can then build a convenient cloning function that can generate multiple layers:
def get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

In [ ]:
class Embedder(nn.Module):
    def __init__(self, vocab_size, d_model):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, d_model)
    def forward(self, x):
        return self.embed(x)

In [ ]:
net=Embedder(100,512)
a=torch.Tensor(1,100)
net(a.type(torch.LongTensor))

tensor([[[ 1.4537, -1.4817, -0.7653,  ...,  0.6204,  1.4153, -0.2327],
         [ 1.4537, -1.4817, -0.7653,  ...,  0.6204,  1.4153, -0.2327],
         [ 1.4537, -1.4817, -0.7653,  ...,  0.6204,  1.4153, -0.2327],
         ...,
         [ 1.4537, -1.4817, -0.7653,  ...,  0.6204,  1.4153, -0.2327],
         [ 1.4537, -1.4817, -0.7653,  ...,  0.6204,  1.4153, -0.2327],
         [ 1.4537, -1.4817, -0.7653,  ...,  0.6204,  1.4153, -0.2327]]],
       grad_fn=<EmbeddingBackward>)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, heads):
        super().__init__()
        self.N = N
        self.embed = Embedder(vocab_size, d_model)
        self.pe = PositionalEncoder(d_model)
        self.layers = get_clones(EncoderLayer(d_model, heads), N)
        self.norm = Norm(d_model)
    def forward(self, src, mask):
        x = self.embed(src)
        x = self.pe(x)
        for i in range(N):
            x = self.layers[i](x, mask)
            
        return self.norm(x)
class Decoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, heads):
        super().__init__()
        self.N = N
        self.embed = Embedder(vocab_size, d_model)
        self.pe = PositionalEncoder(d_model)
        self.layers = get_clones(DecoderLayer(d_model, heads), N)
        self.norm = Norm(d_model)
    def forward(self, trg, e_outputs, src_mask, trg_mask):
        x = self.embed(trg)
        x = self.pe(x)
        for i in range(self.N):
          
          #print(e_outputs)
          x = self.layers[i](x, e_outputs, src_mask, trg_mask)
          #print(x,e_outputs, src_mask, trg_mask)
        return self.norm(x)

In [ ]:
class Transformer(nn.Module):
    def __init__(self, src_vocab, trg_vocab, d_model, N, heads):
        super().__init__()
        self.encoder = Encoder(src_vocab, d_model, N, heads)
        self.decoder = Decoder(trg_vocab, d_model, N, heads)
        self.out = nn.Linear(d_model, trg_vocab)
    def forward(self, src, trg, src_mask, trg_mask):
        e_outputs = self.encoder(src, src_mask)
        #print(e_outputs)
        d_output = self.decoder(trg, e_outputs, src_mask, trg_mask)
        output = self.out(d_output)
        return output

In [ ]:
d_model = 512
heads = 8
N = 6
src_vocab = len(EN_TEXT.vocab)
trg_vocab = len(ES_TEXT.vocab)
model = Transformer(src_vocab, trg_vocab, d_model, N, heads).to(device_gpu)
for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)
optim = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [ ]:
def nopeak_mask(size):
    np_mask = np.triu(np.ones((1, size, size),dtype='uint8'),k=1)
    np_mask =  Variable(torch.from_numpy(np_mask) == 0)
    return np_mask



In [ ]:
def create_masks(src, trg):
    src_pad = EN_TEXT.vocab.stoi['<pad>']
    src_mask = (src != src_pad).unsqueeze(-2)
    src_mask=src_mask.to(device_gpu)

    if trg is not None:
        trg_pad=ES_TEXT.vocab.stoi['<pad>']
        trg_mask = (trg != trg_pad).unsqueeze(-2)
        trg_mask=trg_mask.to(device_gpu)
        size = trg.size(1) # get seq_len for matrix
        
        np_mask = nopeak_mask(size)
        np_mask=np_mask.to(device_gpu)
        trg_mask = trg_mask & np_mask
        
    else:
        trg_mask = None
    return src_mask, trg_mask

In [ ]:
def train_model(epochs, print_every=100):
    
    model.train()
    
    start = time.time()
    temp = start
    
    total_loss = 0
    
    for epoch in range(epochs):
       
        for i, batch in enumerate(train_iter):
            src = batch.English.transpose(0,1)
            trg = batch.Spanish.transpose(0,1)
            
            # the French sentence we input has all words except
            # the last, as it is using each word to predict the next
            
            trg_input = trg[:, :-1]
            
            # the words we are trying to predict
            
            targets = trg[:, 1:].contiguous().view(-1)
            
            # create function to make masks using mask code above
            
            src_mask, trg_mask = create_masks(src, trg_input)
            #print(trg,trg_mask)
            preds = model(src, trg_input, src_mask, trg_mask)
            
            optim.zero_grad()
            
            loss = F.cross_entropy(preds.view(-1, preds.size(-1)),targets)
            loss.backward()
            optim.step()
            
            total_loss += loss.data[0]
            if (i + 1) % print_every == 0:
                loss_avg = total_loss / print_every
                print("time = %dm, epoch %d, iter = %d, loss = %.3f%ds per %d iters" % ((time.time() - start) // 60,
                epoch + 1, i + 1, loss_avg, time.time() - temp,
                print_every))
                total_loss = 0
                temp = time.time()

In [ ]:
import time
from torch.autograd import Variable
t=train_model(epochs=2)

RuntimeError: ignored